In [1]:
#%pip install -U spaCy
#%pip install keras
#%python -m spacy download en_core_web_sm
#%pip install -U jax jaxlib
#%pip install clean-text
#%pip install --upgrade ipykernel
#da eseguire al primo utilizzo

In [1]:
import string
import spacy
import json
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from spacy.lang.it.stop_words import STOP_WORDS
from spacy.lang.en import English
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from pandas import read_csv
from sklearn.svm import SVC
from sklearn import metrics
from sklearn import svm
from sklearn import neural_network
from sklearn import ensemble
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from pandas import read_csv
import nltk
import random

#init spaCy
punctuations = string.punctuation
nlp = spacy.load("en_core_web_lg")
stop_words = spacy.lang.en.stop_words.STOP_WORDS
parser = English()

# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    translator = str.maketrans("", "", string.punctuation)
    text_without_punctuation = text.translate(translator)
    return text_without_punctuation.lower()
    #return text.strip().lower()

# Tokenizer function
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.text for word in mytokens ]
    # remove stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]
    # return preprocessed list of tokens
    return mytokens



In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\perna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
dft = read_csv('../Dataset/Processed datasets/train_sexist.csv')
dfv = read_csv('../Dataset/Processed datasets/dev_sexist.csv')
dfs = read_csv('../Dataset/Processed datasets/test_sexist.csv')

x_train = dft['text']
y_train = dft['label_sexist']
x_val = dfv['text']
y_val = dfv['label_sexist']
x_test = dfs['text']
y_test= dfs['label_sexist']
dft.set_index('ID')
dfv.set_index('ID')
dfs.set_index('ID')
print("TRAIN: \n", y_train.value_counts(), end="\n\n")
print("VALIDATION: \n", y_val.value_counts(), end="\n\n")
print("TEST: \n", y_test.value_counts(), end="\n\n")

TRAIN: 
 not sexist    10602
sexist         3398
Name: label_sexist, dtype: int64

VALIDATION: 
 not sexist    1514
sexist         486
Name: label_sexist, dtype: int64

TEST: 
 not sexist    3030
sexist         970
Name: label_sexist, dtype: int64



In [4]:
vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))

# **Task A**

In [19]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.utils import class_weight
import sklearn.metrics
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, precision_score
from sklearn.neighbors import KNeighborsClassifier


classifier = svm.LinearSVC(max_iter = 10000, class_weight= {"not sexist": 0.2, "sexist": 1})
# Multi-layer Perceptron classifier
#classifier = neural_network.MLPClassifier(hidden_layer_sizes=(512,),verbose=True,max_iter=10)
# Random Foreset
#classifier = ensemble.RandomForestClassifier(class_weight='balanced')

# Create the pipeline

pipe = Pipeline([("cleaner", predictors()),
('vectorizer', vector),
('classifier', classifier)])

pipe.fit(x_train, y_train)


d:\Anaconda\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('cleaner', <__main__.predictors object at 0x000001614C7BC940>),
                ('vectorizer',
                 CountVectorizer(tokenizer=<function spacy_tokenizer at 0x0000016173919CA0>)),
                ('classifier',
                 LinearSVC(class_weight={'not sexist': 0.2, 'sexist': 1},
                           max_iter=10000))])

In [20]:
y_pred = pipe.predict(x_val)

# Calcolo delle metriche di valutazione sul set di test
val_accuracy = accuracy_score(y_val, y_pred)
val_precision = precision_score(y_val, y_pred, average="macro", pos_label="sexist")
val_recall = recall_score(y_val, y_pred, average="macro", pos_label="sexist")
val_f1 = f1_score(y_val, y_pred, average="macro", pos_label="sexist")

# Print the evaluation metrics for the validation data
print("Validation Accuracy: {:.4f}".format(val_accuracy))
print("Validation Precision: {:.4f}".format(val_precision))
print("Validation Recall: {:.4f}".format(val_recall))
print("Validation F1 Score: {:.4f}".format(val_f1))


Validation Accuracy: 0.7830
Validation Precision: 0.7085
Validation Recall: 0.7218
Validation F1 Score: 0.7144


d:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1396: UserWarning: Note that pos_label (set to 'sexist') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
d:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1396: UserWarning: Note that pos_label (set to 'sexist') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
d:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1396: UserWarning: Note that pos_label (set to 'sexist') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


In [7]:
import os
  
n_cpu = os.cpu_count()
print("Number of CPUs in the system:", n_cpu)

Number of CPUs in the system: 8


In [21]:
params = {'vectorizer__ngram_range': [(1,1),(1,2)],
          'classifier__C': [0.4,0.6],
          'classifier__class_weight': [{"not sexist": 0.7, "sexist": 1}, 'balanced', None]}

pipe_optimized = GridSearchCV(pipe,param_grid=params,cv=10,scoring=make_scorer(f1_score, average="macro"),n_jobs=n_cpu-1,refit=True)
pipe_optimized.fit(x_train, y_train)
print("Migliori iperparametri:",pipe_optimized.best_params_)

d:\Anaconda\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Migliori iperparametri: {'classifier__C': 0.4, 'classifier__class_weight': 'balanced', 'vectorizer__ngram_range': (1, 2)}


In [22]:
best_params = pipe_optimized.best_params_
pipe.set_params(**best_params)
pipe.fit(x_train, y_train)

d:\Anaconda\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('cleaner', <__main__.predictors object at 0x000001614C7BC940>),
                ('vectorizer',
                 CountVectorizer(ngram_range=(1, 2),
                                 tokenizer=<function spacy_tokenizer at 0x0000016173919CA0>)),
                ('classifier',
                 LinearSVC(C=0.4, class_weight='balanced', max_iter=10000))])

In [23]:
y_pred = pipe_optimized.predict(x_test)

# Calcolo delle metriche di valutazione sul set di test

val_accuracy = accuracy_score(y_test, y_pred)
val_precision = precision_score(y_test, y_pred, average="macro", pos_label="sexist")
val_recall = recall_score(y_test, y_pred, average="macro", pos_label="sexist")
val_f1 = f1_score(y_test, y_pred, average="macro", pos_label="sexist")

# Print the evaluation metrics for the validation data

print("Validation Accuracy: {:.4f}".format(val_accuracy))
print("Validation Precision: {:.4f}".format(val_precision))
print("Validation Recall: {:.4f}".format(val_recall))
print("Validation F1 Score: {:.4f}".format(val_f1))

Validation Accuracy: 0.8410
Validation Precision: 0.7993
Validation Recall: 0.7384
Validation F1 Score: 0.7606


d:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1396: UserWarning: Note that pos_label (set to 'sexist') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
d:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1396: UserWarning: Note that pos_label (set to 'sexist') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
d:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1396: UserWarning: Note that pos_label (set to 'sexist') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


# **Task B**

In [ ]:
df1 = read_csv('../Dataset/final_dataset_category.csv')
X1 = df1['text']
y1 = df1['label_category']
df1.set_index('ID')
print(y1.value_counts())


2. derogation                               2271
3. animosity                                1665
4. prejudiced discussions                    475
1. threats, plans to harm and incitement     443
Name: label_category, dtype: int64


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

vector1 = CountVectorizer(tokenizer = tokenizer.tokenize, ngram_range=(1,2))

In [ ]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.utils import class_weight
import sklearn.metrics
from sklearn.metrics import precision_score, recall_score, f1_score


#Linear Support Vector Machine classifier
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
accuracy_label = []
precision_label = []
recall_label = []
f1_label = []

class_labels = np.unique(y1)
classifier = svm.LinearSVC(max_iter = 10000)
# Multi-layer Perceptron classifier
#classifier = neural_network.MLPClassifier(hidden_layer_sizes=(512,),verbose=True,max_iter=10)
# Random Foreset
#classifier = ensemble.RandomForestClassifier(class_weight='balanced')

# Create the pipeline

pipe1 = Pipeline([("cleaner", predictors()),
('vectorizer', vector1),
('classifier', classifier)])

kf = KFold(n_splits=10, shuffle=True, random_state=42)

for train_index, test_index in kf.split(X1):
    # Divisione dei dati in fold di addestramento e di test
    X1_train, X1_test = X1[train_index], X1[test_index]
    y1_train, y1_test = y1[train_index], y1[test_index]
    # Addestramento del modello
    pipe1.fit(X1_train, y1_train)

    # Valutazione delle performance del modello sul test set
    predicted = pipe1.predict(X1_test)

    # Calcolo delle metriche di valutazione per il fold corrente
    accuracy = accuracy_score(y1_test, predicted)
    precision = precision_score(y1_test, predicted, average='weighted')
    recall = recall_score(y1_test, predicted, average='weighted')
    f1 = f1_score(y1_test, predicted, average='weighted')

    # Aggiunta delle metriche del fold corrente alle liste
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

    # Calcolo delle metriche per ogni singola classe
    precision_label = precision_score(y1_test, predicted, average=None, labels=class_labels)
    recall_label = recall_score(y1_test, predicted, average=None, labels=class_labels)
    f1_label = f1_score(y1_test, predicted, average=None, labels=class_labels)


In [ ]:
mean_accuracy = np.mean(accuracy_scores)
mean_precision = np.mean(precision_scores)
mean_recall = np.mean(recall_scores)
mean_f1 = np.mean(f1_scores)

# Stampa delle metriche
print("---------------------------------")
print("Metriche finali:")
print(f"Accuratezza media: {mean_accuracy:.4f}")
print(f"Precisione media: {mean_precision:.4f}")
print(f"Richiamo medio: {mean_recall:.4f}")
print(f"F1-score medio: {mean_f1:.4f}")
print("---------------------------------\n")

# Stampa delle metriche per ogni singola classe
print("---------------------------------")
print("Metriche per ogni singola classe:")
for i, label in enumerate(class_labels):
    print(f"Classe '{label}':")
    print(f"Precisione: {precision_label[i]:.4f}")
    print(f"Richiamo: {recall_label[i]:.4f}")
    print(f"F1-score: {f1_label[i]:.4f}")
    print()
print("---------------------------------")

---------------------------------
Metriche finali:
Accuratezza media: 0.5373
Precisione media: 0.5349
Richiamo medio: 0.5373
F1-score medio: 0.5296
---------------------------------

---------------------------------
Metriche per ogni singola classe:
Classe '1. threats, plans to harm and incitement':
Precisione: 0.5714
Richiamo: 0.5333
F1-score: 0.5517

Classe '2. derogation':
Precisione: 0.5783
Richiamo: 0.6761
F1-score: 0.6234

Classe '3. animosity':
Precisione: 0.5183
Richiamo: 0.5000
F1-score: 0.5090

Classe '4. prejudiced discussions':
Precisione: 0.6000
Richiamo: 0.3158
F1-score: 0.4138

---------------------------------


In [ ]:
single_instance = ["I can smell her dying from here."]
prediction = pipe.predict(single_instance)

if (prediction == 'sexist'):
    prediction = pipe1.predict(single_instance)

print(f"Classe: {prediction}")

Classe: ['1. threats, plans to harm and incitement']
